# Todo
* Preprocess the data properly
    - remove frames that has both left hand and right hand as nan
    - Identify the dominant hand NB:(In the current competition, I suspect that both hands would be used in making words or signs)
    - Normalization of the data
    - Pad shorter sequence with 0, and mean pool longer sequence so each video can be of the same frame length. : DONE 
    - Fill NaN values with 0

* How can I map some important signs of the body like
    - lips 
    - nose 
    - pose

* Building of the model I was think of using a GRU model something similar to https://www.kaggle.com/code/nghihuynh/gislr-inference-transformer 
    - Transformer Model 

In [64]:
import os
import cv2
import csv
import pandas as pd
import numpy as np
import mediapipe as mp
import plotly.express as px 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.animation as animation
from IPython.display import HTML


In [10]:
LHAND = np.arange(468, 489).tolist() # 21
RHAND = np.arange(522, 543).tolist() # 21
POSE  = np.arange(489, 522).tolist() # 33
FACE  = np.arange(0,468).tolist()    #468

NOSE=[
    1,2,98,327
]
LNOSE = [98]
RNOSE = [327]
LIP = [ 0, 
    61, 185, 40, 39, 37, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
LLIP = [84,181,91,146,61,185,40,39,37,87,178,88,95,78,191,80,81,82]
RLIP = [314,405,321,375,291,409,270,269,267,317,402,318,324,308,415,310,311,312]

POSE = [500, 502, 504, 501, 503, 505, 512, 513]
LPOSE = [513,505,503,501]
RPOSE = [512,504,502,500]

REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
]
LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
]


In [2]:
data_2 = pd.read_parquet('/Users/dimeji/Documents/Projects/Sign Language/sign_data/asl-signs/train_landmark_files/2044/635217.parquet')

In [3]:
BASE_DIR = 'sign_data/asl-signs'
train = pd.read_csv(f'{BASE_DIR}/train.csv')

In [30]:
train

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [31]:
train

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [32]:
# Get complete file path to file
def get_file_path(path):
    return f'sign_data/asl-signs/{path}'

train['file_path'] = train['path'].apply(get_file_path)

In [33]:
train

,path,participant_id,sequence_id,sign,file_path
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,sign_data/asl-signs/train_landmark_files/26734...
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,sign_data/asl-signs/train_landmark_files/28656...
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,sign_data/asl-signs/train_landmark_files/16069...
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,sign_data/asl-signs/train_landmark_files/25571...
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,sign_data/asl-signs/train_landmark_files/62590...
...,...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white,sign_data/asl-signs/train_landmark_files/53618...
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have,sign_data/asl-signs/train_landmark_files/26734...
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower,sign_data/asl-signs/train_landmark_files/25571...
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room,sign_data/asl-signs/train_landmark_files/29302...


### Ordinal Encode Sign

In [37]:
import json

with open('sign_data/asl-signs/sign_to_prediction_index_map.json') as f: 
    sign_map = json.load(f)

signs = list(sign_map)


In [38]:
signs[:5]

['TV', 'after', 'airplane', 'all', 'alligator']

In [39]:
def mapping_sign_code(sign: str):
    return sign_map[sign]

In [40]:
train['sign_code'] = train['sign'].apply(mapping_sign_code)

In [55]:
from sklearn.preprocessing import LabelEncoder

# Instantiate the encoder
le = LabelEncoder()

# Fit the encoder and transform the 'sign' column
train['sign_code'] = le.fit_transform(train['sign'])

# Create dictionaries for mapping
SIGN2ORD = dict(zip(le.classes_, le.transform(le.classes_)))
ORD2SIGN = dict(zip(le.transform(le.classes_), le.classes_))



In [56]:
display(train.head(10))
display(train.info())

,path,participant_id,sequence_id,sign,file_path,sign_code,sign_ord
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,sign_data/asl-signs/train_landmark_files/26734...,25,25
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,sign_data/asl-signs/train_landmark_files/28656...,232,232
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,sign_data/asl-signs/train_landmark_files/16069...,48,48
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,sign_data/asl-signs/train_landmark_files/25571...,23,23
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,sign_data/asl-signs/train_landmark_files/62590...,164,164
5,train_landmark_files/26734/1000241583.parquet,26734,1000241583,duck,sign_data/asl-signs/train_landmark_files/26734...,67,67
6,train_landmark_files/26734/1000255522.parquet,26734,1000255522,minemy,sign_data/asl-signs/train_landmark_files/26734...,143,143
7,train_landmark_files/32319/1000278229.parquet,32319,1000278229,lips,sign_data/asl-signs/train_landmark_files/32319...,134,134
8,train_landmark_files/37055/100035691.parquet,37055,100035691,flower,sign_data/asl-signs/train_landmark_files/37055...,86,86
9,train_landmark_files/29302/100039661.parquet,29302,100039661,time,sign_data/asl-signs/train_landmark_files/29302...,220,220


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94477 entries, 0 to 94476
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            94477 non-null  object
 1   participant_id  94477 non-null  int64 
 2   sequence_id     94477 non-null  int64 
 3   sign            94477 non-null  object
 4   file_path       94477 non-null  object
 5   sign_code       94477 non-null  int64 
 6   sign_ord        94477 non-null  int16 
dtypes: int16(1), int64(3), object(3)
memory usage: 4.5+ MB


None

In [9]:
ROWS_PER_FRAME = 543  

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [171]:
# Sample 1000 rows
df_sample = train.sample(n=1000, random_state=42)

# List to hold numpy arrays and labels
numpy_arrays = []
labels = []

# Loop over the sampled DataFrame
for _, row in df_sample.iterrows():
    # Load the Parquet file and reshape the data
    array = load_relevant_data_subset(row['file_path'])
    
    # Append numpy array to the list
    numpy_arrays.append(array)
    
    # Append label to the list
    labels.append(row['sign_ord'])

# Convert lists to numpy arrays
numpy_arrays = np.array(numpy_arrays)
labels = np.array(labels)

sign_data/asl-signs/train_landmark_files/28656/3311214787.parquet
sign_data/asl-signs/train_landmark_files/53618/3588192588.parquet
sign_data/asl-signs/train_landmark_files/4718/1363575346.parquet
sign_data/asl-signs/train_landmark_files/37779/951199059.parquet
sign_data/asl-signs/train_landmark_files/36257/283190141.parquet
sign_data/asl-signs/train_landmark_files/61333/186594661.parquet
sign_data/asl-signs/train_landmark_files/53618/782770724.parquet
sign_data/asl-signs/train_landmark_files/16069/2977903115.parquet
sign_data/asl-signs/train_landmark_files/2044/269101282.parquet
sign_data/asl-signs/train_landmark_files/28656/3171133897.parquet
sign_data/asl-signs/train_landmark_files/49445/2425523049.parquet
sign_data/asl-signs/train_landmark_files/18796/3586377265.parquet
sign_data/asl-signs/train_landmark_files/34503/98693993.parquet
sign_data/asl-signs/train_landmark_files/2044/1051321002.parquet
sign_data/asl-signs/train_landmark_files/4718/3910863104.parquet
sign_data/asl-signs/t

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1000,) + inhomogeneous part.

In [172]:
numpy_arrays[0][0][522:542]


array([[ 1.5144347e-01,  7.5613767e-01,  1.2740776e-06],
       [ 2.4153522e-01,  7.2649568e-01, -2.2481397e-02],
       [ 2.9584736e-01,  6.6692662e-01, -3.0620238e-02],
       [ 3.1995028e-01,  6.1712366e-01, -4.3287758e-02],
       [ 3.4872800e-01,  5.7083178e-01, -5.5275746e-02],
       [ 2.3215996e-01,  5.8595455e-01,  5.2190065e-02],
       [ 2.5217044e-01,  5.2832663e-01,  6.2967956e-02],
       [ 2.7134955e-01,  4.8986065e-01,  5.9470192e-02],
       [ 2.8328773e-01,  4.5986822e-01,  5.1828388e-02],
       [ 1.8974334e-01,  5.7735515e-01,  4.7869675e-02],
       [ 2.2290841e-01,  5.2601856e-01,  4.7441982e-02],
       [ 2.6852223e-01,  5.0664395e-01,  2.9607691e-02],
       [ 3.1005228e-01,  5.0007749e-01,  1.9713828e-02],
       [ 1.3828482e-01,  5.7924539e-01,  3.5692640e-02],
       [ 1.4157811e-01,  5.1354206e-01,  3.2111593e-02],
       [ 1.5732832e-01,  4.7189856e-01,  9.0731150e-03],
       [ 1.7816862e-01,  4.4048345e-01, -8.0074295e-03],
       [ 8.2876854e-02,  5.9825

In [196]:
a = 0
for i in range(len(numpy_arrays)): 
    a += len(numpy_arrays[i])
print(a/len(range(i)))

38.42042042042042


In [200]:
len(range(i))

999

In [201]:
a

38382

In [134]:
labels[:10]



[206, 20, 178, 114, 221, 230, 25, 236, 184, 191]

In [144]:
numpy_arrays[0][0][522:542]

array([[ 1.5144347e-01,  7.5613767e-01,  1.2740776e-06],
       [ 2.4153522e-01,  7.2649568e-01, -2.2481397e-02],
       [ 2.9584736e-01,  6.6692662e-01, -3.0620238e-02],
       [ 3.1995028e-01,  6.1712366e-01, -4.3287758e-02],
       [ 3.4872800e-01,  5.7083178e-01, -5.5275746e-02],
       [ 2.3215996e-01,  5.8595455e-01,  5.2190065e-02],
       [ 2.5217044e-01,  5.2832663e-01,  6.2967956e-02],
       [ 2.7134955e-01,  4.8986065e-01,  5.9470192e-02],
       [ 2.8328773e-01,  4.5986822e-01,  5.1828388e-02],
       [ 1.8974334e-01,  5.7735515e-01,  4.7869675e-02],
       [ 2.2290841e-01,  5.2601856e-01,  4.7441982e-02],
       [ 2.6852223e-01,  5.0664395e-01,  2.9607691e-02],
       [ 3.1005228e-01,  5.0007749e-01,  1.9713828e-02],
       [ 1.3828482e-01,  5.7924539e-01,  3.5692640e-02],
       [ 1.4157811e-01,  5.1354206e-01,  3.2111593e-02],
       [ 1.5732832e-01,  4.7189856e-01,  9.0731150e-03],
       [ 1.7816862e-01,  4.4048345e-01, -8.0074295e-03],
       [ 8.2876854e-02,  5.9825

In [170]:
for k in range(len(numpy_arrays)):
    frame_mask = np.logical_or(~np.isnan(numpy_arrays[k][:, 468:488]).all(axis=(1, 2)), 
                               ~np.isnan(numpy_arrays[k][:, 522:542]).all(axis=(1, 2)))
    numpy_arrays[k] = numpy_arrays[k][frame_mask]

In [166]:
def pad_edge(t, repeats, side):
    if side == 'LEFT':
        return tf.concat((tf.repeat(t[:1], repeats=repeats, axis=0), t), axis=0)
    elif side == 'RIGHT':
        return tf.concat((t, tf.repeat(t[-1:], repeats=repeats, axis=0)), axis=0)


20

In [175]:
def adjust_sequence_tf(data, input_size):
    # Get the sequence length (assuming it's the first dimension)
    seq_len = tf.shape(data)[0]

    # Remove frames where both hands have all NaN values
    left_hand_mask = tf.reduce_all(tf.math.is_nan(data[:, 468:488]), axis=[1,2])
    right_hand_mask = tf.reduce_all(tf.math.is_nan(data[:, 522:542]), axis=[1,2])
    both_hands_nan_mask = tf.logical_and(left_hand_mask, right_hand_mask)
    valid_frames_mask = tf.logical_not(both_hands_nan_mask)
    data = tf.boolean_mask(data, valid_frames_mask, axis=0)

    # Update the sequence length after removing NaN frames
    seq_len = tf.shape(data)[0]

    # Remove frames where mean of landmarks is 0
    mean_landmarks_per_frame = tf.reduce_mean(data, axis=[1,2])
    non_zero_mean_mask = tf.not_equal(mean_landmarks_per_frame, 0.0)
    data = tf.boolean_mask(data, non_zero_mean_mask, axis=0)

    # Update the sequence length after removing zero-mean frames
    seq_len = tf.shape(data)[0]

    if seq_len < input_size:
        # If sequence is shorter than input_size, pad it with edge values
        padding_size = input_size - seq_len
        left_pad = padding_size // 2
        right_pad = padding_size - left_pad
        data = pad_edge(data, left_pad, 'LEFT')
        data = pad_edge(data, right_pad, 'RIGHT')
    elif seq_len > input_size:
        # If sequence is longer than input_size, use mean pooling to downsample it
        # If sequence length is less than square of input_size, repeat the frames
        if seq_len < input_size ** 2:
            repeats = input_size * input_size // seq_len
            data = tf.repeat(data, repeats, axis=0)
            seq_len = tf.shape(data)[0]  # update sequence length

        # Pad to multiple of input_size
        pad_size = -seq_len % input_size  # the amount to pad to make the length a multiple of input_size
        pad_left = pad_size // 2
        pad_right = pad_size - pad_left
        data = pad_edge(data, pad_left, 'LEFT')
        data = pad_edge(data, pad_right, 'RIGHT')

        # Mean Pool
        pool_size = tf.shape(data)[0] // input_size
        data = tf.reshape(data, [input_size, pool_size] + data.shape[1:].as_list())
        data = tf.reduce_mean(data, axis=1)

    data = tf.where(tf.math.is_nan(data), 0.0, data)
    
    return data

# Set desired input size
input_size = 100  # Replace with your desired input size

# Adjust sequences
# Assuming `tf_arrays` is a list of your data arrays in tf.Tensor format
tf_arrays_adj = [adjust_sequence_tf(arr, input_size) for arr in numpy_arrays]


In [182]:
len(tf_arrays_adj[2][2])

543